# HR Analytics

<img src = 'https://datahack-prod.s3.ap-south-1.amazonaws.com/__sized__/contest_cover/hr_1920x480_s5WuoZs-thumbnail-1200x1200-90.jpg'>

Practice Problem: https://datahack.analyticsvidhya.com/contest/wns-analytics-hackathon-2018-1/

## HR Analytics

HR analytics is revolutionising the way human resources departments operate, leading to higher efficiency and better results overall. Human resources has been using analytics for years. However, the collection, processing and analysis of data has been largely manual, and given the nature of human resources dynamics and HR KPIs, the approach has been constraining HR. Therefore, it is surprising that HR departments woke up to the utility of machine learning so late in the game. Here is an opportunity to try predictive analytics in identifying the employees most likely to get promoted.

## Problem Statement

Your client is a large MNC and they have 9 broad verticals across the organisation. One of the problem your client is facing is around identifying the right people for promotion *(only for manager position and below)* and prepare them in time. Currently the process, they are following is:

* They first identify a set of employees based on recommendations/ past performance
* Selected employees go through the separate training and evaluation program for each vertical. These programs are based on the required skill of each vertical
* At the end of the program, based on various factors such as training performance, KPI completion (only employees with KPIs completed greater than 60% are considered) etc., employee gets promotion

For above mentioned process, the final promotions are only announced after the evaluation and this leads to delay in transition to their new roles. Hence, company needs your help in identifying the eligible candidates at a particular checkpoint so that they can expedite the entire promotion cycle. 

<img src = 'https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/09/wns_hack_im_1.jpg'>

They have provided multiple attributes around Employee's past and current performance along with demographics. Now, The task is to predict whether a potential promotee at checkpoint in the test set will be promoted or not after the evaluation process.

## Evaluation Metric

The evaluation metric for this competition is F1 Score.

## Public and Private Split

Test data is further randomly divided into Public (40%) and Private (60%) data.

Your initial responses will be checked and scored on the Public data.
The final rankings would be based on your private score which will be published once the competition is over.

## Entorno

In [1]:
import sys
sys.version

'3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]'

In [2]:
!conda info --envs

# conda environments:
#
micromaster              /Users/manuel/.conda/envs/micromaster
                         /Users/manuel/.julia/conda/3
base                  *  /Users/manuel/opt/anaconda3
belcorp                  /Users/manuel/opt/anaconda3/envs/belcorp
courseragcp              /Users/manuel/opt/anaconda3/envs/courseragcp
iapucp                   /Users/manuel/opt/anaconda3/envs/iapucp
mitxpro                  /Users/manuel/opt/anaconda3/envs/mitxpro
style-transfer           /Users/manuel/opt/anaconda3/envs/style-transfer
taller-dmc               /Users/manuel/opt/anaconda3/envs/taller-dmc
udacity                  /Users/manuel/opt/anaconda3/envs/udacity



## Paquetes

In [33]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
from pathlib import Path
import random
import warnings
import pickle

warnings.filterwarnings('ignore')


seed = 2020
random.seed(seed)

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 400)
sns.set()

DATA = Path('../../data') 
RAW  = DATA/'raw'
PROCESSED = DATA/'processed'
SUBMISSIONS = DATA/'submissions'    

MODEL = Path('../../model') 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
pd.__version__

'1.1.3'

In [35]:
np.__version__

'1.19.2'

In [36]:
sklearn.__version__

'0.23.2'

In [37]:
id_columns = 'employee_id'
target = 'is_promoted'

## Lectura de datos

In [38]:
os.listdir(f'{PROCESSED}')

['.DS_Store',
 'preprocess_v1_capping_values.pkl',
 'preprocess_v1_impute_values.pkl',
 'preprocess_v1_ohe.pkl',
 'preprocess_v1_ohe_columns.pkl',
 'preprocess_v1_over50_train.csv',
 'preprocess_v1_scaler.pkl',
 'preprocess_v1_smote20_train.csv',
 'preprocess_v1_smote50_train.csv',
 'preprocess_v1_smoteTomek20_train.csv',
 'preprocess_v1_smoteTomek50_train.csv',
 'preprocess_v1_train.csv',
 'preprocess_v1_under50_train.csv',
 'preprocess_v1_val.csv',
 'preprocess_v2_capping_values.pkl',
 'preprocess_v2_knnimputation.pkl',
 'preprocess_v2_ohe.pkl',
 'preprocess_v2_ohe_columns.pkl',
 'preprocess_v2_over50_train.csv',
 'preprocess_v2_scaler.pkl',
 'preprocess_v2_scalerimputation.pkl',
 'preprocess_v2_smote20_train.csv',
 'preprocess_v2_smote50_train.csv',
 'preprocess_v2_smoteTomek20_train.csv',
 'preprocess_v2_smoteTomek50_train.csv',
 'preprocess_v2_test.csv',
 'preprocess_v2_train.csv',
 'preprocess_v2_under50_train.csv',
 'preprocess_v2_val.csv']

## Entrenamiento V1 sin balanceo

In [39]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import precision_recall_curve, roc_auc_score, f1_score

In [40]:
preproc_train = [file for file in os.listdir(f'{PROCESSED}') if file.endswith('train.csv')]
preproc_train

['preprocess_v1_over50_train.csv',
 'preprocess_v1_smote20_train.csv',
 'preprocess_v1_smote50_train.csv',
 'preprocess_v1_smoteTomek20_train.csv',
 'preprocess_v1_smoteTomek50_train.csv',
 'preprocess_v1_train.csv',
 'preprocess_v1_under50_train.csv',
 'preprocess_v2_over50_train.csv',
 'preprocess_v2_smote20_train.csv',
 'preprocess_v2_smote50_train.csv',
 'preprocess_v2_smoteTomek20_train.csv',
 'preprocess_v2_smoteTomek50_train.csv',
 'preprocess_v2_train.csv',
 'preprocess_v2_under50_train.csv']

In [41]:
preproc_val = [file for file in os.listdir(f'{PROCESSED}') if file.endswith('val.csv')]
preproc_val

['preprocess_v1_val.csv', 'preprocess_v2_val.csv']

In [42]:
for train_file in sorted(preproc_train):
    df_train = pd.read_csv(f'{PROCESSED}/{train_file}', compression = 'zip')
    df_val = pd.read_csv(f'{PROCESSED}/{preproc_val[0]}', compression = 'zip')
    
    print(f'label: {train_file:35} \tnrows: {len(df_train)} \t%target train: {df_train[target].mean():.4f} \t%target val: {df_val[target].mean():.4f}')

label: preprocess_v1_over50_train.csv      	nrows: 80224 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v1_smote20_train.csv     	nrows: 48134 	%target train: 0.1667 	%target val: 0.0852
label: preprocess_v1_smote50_train.csv     	nrows: 80224 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v1_smoteTomek20_train.csv 	nrows: 46412 	%target train: 0.1543 	%target val: 0.0852
label: preprocess_v1_smoteTomek50_train.csv 	nrows: 79638 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v1_train.csv             	nrows: 43846 	%target train: 0.0852 	%target val: 0.0852
label: preprocess_v1_under50_train.csv     	nrows: 7468 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v2_over50_train.csv      	nrows: 80224 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v2_smote20_train.csv     	nrows: 48134 	%target train: 0.1667 	%target val: 0.0852
label: preprocess_v2_smote50_train.csv     	nrows: 80224 	%target train: 0.5000 	%target v

In [44]:
train_file = 'preprocess_v1_train.csv'
val_file = 'preprocess_v1_val.csv'


In [45]:
from sklearn.model_selection import ParameterGrid

In [47]:
cv_grid = {'penalty': ['l1','l2'],'solver': ['liblinear','saga'],
              'C': [0.001,0.01,0.1,1,10,100,1000],
            'random_state': [seed]}

params_grid = list(ParameterGrid(cv_grid))

In [48]:
df_results = pd.DataFrame(columns = ['preproc_label', 'model_label', 'método', 'parámetros', 'columnas_out',
                                     'auc_train', 'auc_val', 'threshold','f1_train', 'f1_val'])


for xgb_params in tqdm(params_grid):
    
    for train_file in sorted(preproc_train):

        preproc_label = train_file.split('_train')[0]

        print('----------------------------------------------------------------------')
        print(xgb_params)
        print(train_file)
        print('----------------------------------------------------------------------')

        df_train = pd.read_csv(f'{PROCESSED}/{train_file}', compression = 'zip')
        df_val = pd.read_csv(f'{PROCESSED}/{preproc_val[0]}', compression = 'zip')

        X_train, y_train = df_train.drop(target, axis = 1), df_train[target]
        X_val, y_val = df_val.drop(target, axis = 1), df_val[target]

        logi = LogisticRegression(solver = xgb_params["solver"],penalty =xgb_params["penalty"], C =xgb_params["C"], random_state= xgb_params["random_state"])
        logi_fit = logi.fit( X_train,y_train )
                        
        
        #xgb_params_export = xgb_params.copy()
        #xgb_params_export.update(logi_fit.attributes())

        probs_train = logi_fit.predict(X_train)
        probs_val = logi_fit.predict(X_val)

        auc_train = roc_auc_score(y_train, probs_train)
        auc_val = roc_auc_score(y_val, probs_val)

        #best threshold
        prec, recall, threshold = precision_recall_curve(y_train, probs_train)
        prec_recall = pd.DataFrame({'prec': prec[:-1], 'recall': recall[:-1], 'threshold': threshold})
        prec_recall['f1'] = 2*prec_recall['prec']*prec_recall['recall'] / (prec_recall['prec'] + prec_recall['recall'])
        prec_recall = prec_recall.sort_values(by = 'f1', ascending = False).head(1)

        #f1 scores
        best_threshold = prec_recall['threshold'].values[0]
        f1_train = prec_recall['f1'].values[0]

        labels_val = np.where(probs_val >= best_threshold, 1, 0)
        f1_val = f1_score(y_val, labels_val)

        print(f'auc_train: {auc_train:.6f} \tauc_val: {auc_val:.6f} \tf1_train: {f1_train:.6f} \tf1_val: {f1_val:.6f}')

        results = [preproc_label, 'LogisticRegression', 'fit', xgb_params, '',
                  auc_train, auc_val, best_threshold, f1_train, f1_val]


        df_results.loc[len(df_results)] = results

  0%|          | 0/28 [00:00<?, ?it/s]

----------------------------------------------------------------------
{'C': 0.001, 'penalty': 'l1', 'random_state': 2020, 'solver': 'liblinear'}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.772961 	auc_val: 0.779456 	f1_train: 0.784061 	f1_val: 0.346537
----------------------------------------------------------------------
{'C': 0.001, 'penalty': 'l1', 'random_state': 2020, 'solver': 'liblinear'}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.543655 	auc_val: 0.551785 	f1_train: 0.285704 	f1_val: 0.157028
----------------------------------------------------------------------
{'C': 0.001, 'penalty': 'l1', 'random_state': 2020, 'solver': 'liblinear'}
preprocess_v1_smote50_train.csv
----------------------------------------------------------------------
auc_train: 0.788791 	auc_val: 0.781288 	f1_train: 0.802107 	f1_val: 0.348910
-----------------------

  4%|▎         | 1/28 [00:18<08:23, 18.66s/it]

auc_train: 0.713846 	auc_val: 0.712636 	f1_train: 0.723866 	f1_val: 0.286592
----------------------------------------------------------------------
{'C': 0.001, 'penalty': 'l1', 'random_state': 2020, 'solver': 'saga'}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.766117 	auc_val: 0.777858 	f1_train: 0.773844 	f1_val: 0.351086
----------------------------------------------------------------------
{'C': 0.001, 'penalty': 'l1', 'random_state': 2020, 'solver': 'saga'}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.540414 	auc_val: 0.548337 	f1_train: 0.285704 	f1_val: 0.157028
----------------------------------------------------------------------
{'C': 0.001, 'penalty': 'l1', 'random_state': 2020, 'solver': 'saga'}
preprocess_v1_smote50_train.csv
----------------------------------------------------------------------
auc_train: 0.782235 	auc_val: 0.772940

  7%|▋         | 2/28 [00:40<08:33, 19.74s/it]

...

auc_train: 0.790037 	auc_val: 0.799382 	f1_train: 0.796046 	f1_val: 0.386307
----------------------------------------------------------------------
{'C': 1000, 'penalty': 'l2', 'random_state': 2020, 'solver': 'liblinear'}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.791235 	auc_val: 0.794847 	f1_train: 0.797362 	f1_val: 0.380129
----------------------------------------------------------------------
{'C': 1000, 'penalty': 'l2', 'random_state': 2020, 'solver': 'liblinear'}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.674508 	auc_val: 0.685453 	f1_train: 0.493065 	f1_val: 0.457904
----------------------------------------------------------------------
{'C': 1000, 'penalty': 'l2', 'random_state': 2020, 'solver': 'liblinear'}
preprocess_v1_smote50_train.csv
----------------------------------------------------------------------
auc_train: 0.806517 	auc_v

 96%|█████████▋| 27/28 [22:01<00:35, 35.51s/it]

auc_train: 0.789770 	auc_val: 0.799332 	f1_train: 0.795733 	f1_val: 0.386210
----------------------------------------------------------------------
{'C': 1000, 'penalty': 'l2', 'random_state': 2020, 'solver': 'saga'}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.791309 	auc_val: 0.794847 	f1_train: 0.797455 	f1_val: 0.380129
----------------------------------------------------------------------
{'C': 1000, 'penalty': 'l2', 'random_state': 2020, 'solver': 'saga'}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.674508 	auc_val: 0.685453 	f1_train: 0.493065 	f1_val: 0.457904
----------------------------------------------------------------------
{'C': 1000, 'penalty': 'l2', 'random_state': 2020, 'solver': 'saga'}
preprocess_v1_smote50_train.csv
----------------------------------------------------------------------
auc_train: 0.806517 	auc_val: 0.796498 	f

100%|██████████| 28/28 [22:35<00:00, 48.42s/it]

auc_train: 0.790037 	auc_val: 0.799382 	f1_train: 0.796046 	f1_val: 0.386307


In [49]:
df_results

,preproc_label,model_label,método,parámetros,columnas_out,auc_train,auc_val,threshold,f1_train,f1_val
0,preprocess_v1_over50,xgboost_baseline,fit,"{'C': 0.001, 'penalty': 'l1', 'random_state': ...",,0.772961,0.779456,1,0.784061,0.346537
1,preprocess_v1_smote20,xgboost_baseline,fit,"{'C': 0.001, 'penalty': 'l1', 'random_state': ...",,0.543655,0.551785,0,0.285704,0.157028
2,preprocess_v1_smote50,xgboost_baseline,fit,"{'C': 0.001, 'penalty': 'l1', 'random_state': ...",,0.788791,0.781288,1,0.802107,0.348910
3,preprocess_v1_smoteTomek20,xgboost_baseline,fit,"{'C': 0.001, 'penalty': 'l1', 'random_state': ...",,0.540606,0.548873,0,0.267336,0.157028
4,preprocess_v1_smoteTomek50,xgboost_baseline,fit,"{'C': 0.001, 'penalty': 'l1', 'random_state': ...",,0.792838,0.782408,1,0.805856,0.349722


In [51]:
MODELS = DATA/'models'

In [52]:
df_results.to_csv(f'{MODELS}/logistic_base.csv', index = False)

In [53]:
df_results.to_csv('logistic_base.csv')

In [58]:
df_results.to_excel(f'{MODELS}/logistic_base.xlsx', index = False)